In [151]:
%load_ext autoreload
%autoreload 2

import formatOrtho
import pyhamPipeline

import pyham
from pyoma.browser import db 
import numpy as np
from tables import *
import re
from ete3 import Tree
import pickle
import tempfile
import functools
import config

h5file = open_file(config.omadirLaurent + 'OmaServer.h5', mode="r")
dbObj = db.Database(h5file)
omaIdObj = db.OmaIdMapper(dbObj)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/laurent/anaconda3/lib/python3.6/site-packages/tables/leaf.py:396: PerformanceWarning: The Leaf ``/Protein/_i_Entries/OmaHOG/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


In [152]:
species_tree, replacement_dico = formatOrtho.fix_species_tree("speciestree.nwk", omaIdObj)

name
LUCA
Nanosalinarum sp. (strain J07AB56)
root name
LUCA
123851


In [153]:
ortho = dbObj.get_orthoxml(100000).decode()

In [154]:
newortho = formatOrtho.convert_orthoxml_ids(ortho, replacement_dico)

In [155]:
hamObj = pyham.Ham(species_tree, newortho, type_hog_file="string", use_internal_name = True)

In [156]:
print(hamObj.get_dict_top_level_hogs())


{'100000': <HOG(100000)>}


In [157]:
hog21 = hamObj.get_hog_by_id('100000')
tp = hamObj.create_tree_profile(hog=hog21, outfile='bite.png')

In [158]:
tp.treemap

Tree node 'rosids' (0x7f026412d6a)

In [159]:

for node in tp.treemap.traverse():
    if not node.is_root():
        print(node.name)
        print('N')
        print(node.nbr_genes)
        print('D')
        print(node.dupl)
        print('L')
        print(node.lost)
        print('I')
        print(node.identical)


fabids
N
1
D
0
L
0
I
1
malvids
N
1
D
0
L
0
I
1
Vitis vinifera
N
0
D
0
L
1
I
0
Papilionoideae
N
1
D
0
L
0
I
1
Malpighiales
N
0
D
0
L
1
I
0
Prunus persica
N
0
D
0
L
1
I
0
Brassicaceae
N
1
D
0
L
0
I
1
Malvaceae
N
0
D
0
L
1
I
0
Lotus japonicus
N
0
D
0
L
1
I
0
Glycine max
N
0
D
0
L
1
I
0
Medicago truncatula
N
1
D
0
L
0
I
1
Manihot esculenta
N
0
D
0
L
0
I
0
Populus trichocarpa
N
0
D
0
L
0
I
0
Brassica
N
1
D
0
L
0
I
1
Arabidopsis
N
0
D
0
L
1
I
0
Theobroma cacao
N
0
D
0
L
0
I
0
Gossypium hirsutum
N
0
D
0
L
0
I
0
Brassica napus
N
1
D
0
L
0
I
1
Brassica oleracea
N
1
D
0
L
0
I
1
Arabidopsis lyrata
N
0
D
0
L
0
I
0
Arabidopsis thaliana
N
0
D
0
L
0
I
0


In [160]:
T = Tree(species_tree , format =1 )


In [161]:
mapping = {}
mappingINV = {}
for i,node in enumerate(T.traverse()):
    mapping[i] = node.name
    mappingINV[node.name] = i


In [162]:
import datasketch
import ete3
import sparse
import itertools

In [163]:
def Tree2Hashes(treemap):
    #turn each tree into a minhash object
    #serialize and store as array



    eventdict = { 'presence':[] , 'gain':[] , 'loss':[] , 'duplication':[]}


    for node in treemap.traverse():
    # traverse() returns an iterator to traverse the tree structure
    # strategy:"levelorder" by default; nodes are visited in order from root to leaves
    # it return treeNode instances
        if not node.is_root():
            print(node.name)
            if node.nbr_genes >0:
                eventdict['presence'].append('P'+node.name)
            if node.dupl > 0:
                eventdict['duplication'].append('D'+node.name)
            if node.lost > 0:
                eventdict['loss'].append('L'+node.name)
        else:
            eventdict['gain'].append('G'+node.name)

        hashes= []

    for array in eventdict:
        #generate minhash
        # why set ? 
        eventdict[array] = set(eventdict[array])
        m1 = datasketch.MinHash(num_perm=128)
        for element in eventdict[array]:
            m1.update(element.encode())
        m1 = datasketch.LeanMinHash(m1)
        buf = bytearray(m1.bytesize())
        m1.serialize(buf)
        print(buf)
        hashes.append([buf])
    hashmat = np.vstack(hashes)
    return hashmat

hashmat = Tree2Hashes(tp.treemap)
print(hashmat.shape)
print(hashmat)

fabids
malvids
Vitis vinifera
Papilionoideae
Malpighiales
Prunus persica
Brassicaceae
Malvaceae
Lotus japonicus
Glycine max
Medicago truncatula
Manihot esculenta
Populus trichocarpa
Brassica
Arabidopsis
Theobroma cacao
Gossypium hirsutum
Brassica napus
Brassica oleracea
Arabidopsis lyrata
Arabidopsis thaliana
bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x18\x16z\x02\x91\xf9\x13\x1fm\xe1\xc2\x0e\xa1\xdec:\x10\x9d]\x0b\x1d0\x125\xbb*\x84+\xec\xc7\xb5\x1f\x16\'\xa4\x1e\xcf\xc28\x18ti\x0c\x01{\\"\x01\xf9\xdb\xad/)\x1f\xbeU.\xd0~\r\xcb \x8d\x13t\xfe\xf7\x1dcZ\xf3\x19W+\x0c\x04k\xa5\xbc\x08\xb8\xfb\xe6\x0c\x08\xc6\x11\x1a\xdd\xfdX1\xb4\x0e\x9c\x03`\xdd\x14*\x04\xfb\x1d0\xda\x1e\x98\x10\x97\xed6\x1a=/\xfc\x04\x04F\x03\n\xe6\xb5\xbf\x1c\xa5\xee%\x1dK&\x8c\rI&T\x05Tc\xf2\x10\xe5\xbe\xe3\x07\xef:\'\x01au\x0e\x0fW(\xee\x00U\xd9a\x06\xa3B\x9bK\xd0\x8b\x93\x00\xcf\x92\xe4\x10\xb7\xbcT\x05\x84s>\x05\x93M{\tX\xaf\xbe\x10h\xab\xb4 =\xedc\x0f\x93\xc9l\x1e\xf8\xa3\xb9\x0c\x19\xc5#\x16\x1

In [178]:
import numpy as np
from scipy.sparse import csr_matrix

def Tree2mat(eteobj,taxaIndex):
    #use partials to configure the taxa index
    #turn each tree into a sparse matrix with 4 rows
    rowdict={ 'presence':0 , 'gain':1 , 'loss':2 , 'duplication':3}
    
    mat = csr_matrix( (len(rowdict), len(taxaIndex) ) )
    
    coords = []
    for node in eteobj.traverse():
    # traverse() returns an iterator to traverse the tree structure
    # strategy:"levelorder" by default; nodes are visited in order from root to leaves
    # it return treeNode instances
        if not node.is_root():            
            if node.nbr_genes >0:
                mat[ rowdict['presence'] , taxaIndex[node.name]] =1 
            if node.lost > 0:
                mat[ rowdict['loss'] , taxaIndex[node.name]] =1
            if node.dupl > 0:
                mat[ rowdict['duplication'] , taxaIndex[node.name]] =1
        else:
            mat[ rowdict['gain'] , taxaIndex[node.name]] =1
  
    return mat

print(np.sum(Tree2mat(tp.treemap, mappingINV)))



16.0


/home/laurent/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
